[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/accdavlo/calcolo-scientifico/blob/main/codes/ROM_with_FEniCS.ipynb)

In [ ]:
# Installing FEniCS (dolfin) on the Google Colab servers
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin
from dolfin import *

In [ ]:
# Setting some plotting styles
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12, 9)

In [ ]:
# Importing some libraries
from dolfin import * # This is the core of FEniCS and it contains all the FEM functions we will need
from ufl_legacy.geometry import * # This helps in designing geometries
from dolfin.cpp.mesh import *     # This handles meshes
from mshr import *                # This generates meshes

# ROM

In the case of parametric problems where we are interested in the solution for many different parameters or in a very fast evaluation of the map parameter-to-solution, we might interested in using reduced order models.

Let's start with an example taken from the [RBniCS library](https://www.rbnicsproject.org/), a reduced basis library based on FEniCS.

## TUTORIAL 01 - Thermal block problem
**_Keywords: certified reduced basis method, scalar problem_**

### 1. Introduction
In this Tutorial, we consider steady heat conduction in a two-dimensional domain $\Omega=[0,1]^2$.

<img src="https://github.com/RBniCS/RBniCS/raw/master/tutorials/01_thermal_block/data/thermal_block.png" />

We define two subdomains $\Omega_1$ and $\Omega_2$, such that
1. $\Omega_1$ is a disk centered at the origin of radius $r_0=0.25$, and
2. $\Omega_2=\Omega/\ \overline{\Omega_1}$.

The conductivity $\kappa$ is assumed to be constant on $\Omega_1$ and $\Omega_2$, i.e.
$$
\kappa|_{\Omega_1}=\kappa_0 \quad \textrm{and} \quad \kappa|_{\Omega_2}=1.
$$

For this problem, we consider $P=2$ parameters:
1. the first one is related to the conductivity in $\Omega_1$, i.e. $\mu_0\equiv k_0$ (_note that parameters numbering is zero-based_);
2. the second parameter $\mu_1$ takes into account the constant heat flux over $\Gamma_{base}$.

The parameter vector $\boldsymbol{\mu}$ is thus given by
$$
\boldsymbol{\mu} = (\mu_0,\mu_1)
$$
on the parameter domain
$$
\mathbb{P}=[0.1,10]\times[-1,1].
$$

In this problem we model the heat transfer process due to the heat flux over the bottom boundary $\Gamma_{base}$ and the following conditions on the remaining boundaries:
* the left and right boundaries $\Gamma_{side}$ are insulated,
* the top boundary $\Gamma_{top}$ is kept at a reference temperature (say, zero).

In order to obtain a faster evaluation (yet, provably accurate) of the output of interest we propose to use a certified reduced basis approximation for the problem.

### 2. Parametrized formulation

Let $u(\boldsymbol{\mu})$ be the temperature in the domain $\Omega$.

The strong formulation of the parametrized problem is given by:
for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u(\boldsymbol{\mu})$ such that

$$
\begin{cases}
	- \text{div} (\kappa(\mu_0)\nabla u(\boldsymbol{\mu})) = 0 & \text{in } \Omega,\\
	u(\boldsymbol{\mu}) = 0 & \text{on } \Gamma_{top},\\
	\kappa(\mu_0)\nabla u(\boldsymbol{\mu})\cdot \mathbf{n} = 0 & \text{on } \Gamma_{side},\\
	\kappa(\mu_0)\nabla u(\boldsymbol{\mu})\cdot \mathbf{n} = \mu_1 & \text{on } \Gamma_{base}.
\end{cases}
$$
<br>

where
* $\mathbf{n}$ denotes the outer normal to the boundaries $\Gamma_{side}$ and $\Gamma_{base}$,
* the conductivity $\kappa(\mu_0)$ is defined as follows:
$$
\kappa(\mu_0) =
\begin{cases}
	\mu_0 & \text{in } \Omega_1,\\
	1 & \text{in } \Omega_2,\\
\end{cases}
$$

The corresponding weak formulation reads:
for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u(\boldsymbol{\mu})\in\mathbb{V}$ such that

$$a\left(u(\boldsymbol{\mu}),v;\boldsymbol{\mu}\right)=f(v;\boldsymbol{\mu})\quad \forall v\in\mathbb{V}$$

where

* the function space $\mathbb{V}$ is defined as
$$
\mathbb{V} = \{v\in H^1(\Omega) : v|_{\Gamma_{top}}=0\}
$$
* the parametrized bilinear form $a(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$a(u, v;\boldsymbol{\mu})=\int_{\Omega} \kappa(\mu_0)\nabla u\cdot \nabla v \ d\boldsymbol{x},$$
* the parametrized linear form $f(\cdot; \boldsymbol{\mu}): \mathbb{V} \to \mathbb{R}$ is defined by
$$f(v; \boldsymbol{\mu})= \mu_1\int_{\Gamma_{base}}v \ ds.$$

The (compliant) output of interest $s(\boldsymbol{\mu})$ given by
$$s(\boldsymbol{\mu}) = \mu_1\int_{\Gamma_{base}} u(\boldsymbol{\mu})$$
is computed for each $\boldsymbol{\mu}$.

## 3. Affine decomposition

For this problem the affine decomposition is straightforward:
$$a(u,v;\boldsymbol{\mu})=  \underbrace{1}_{\Theta^{a}_0(\boldsymbol{\mu})}\underbrace{\int_{\Omega_2}\nabla u \cdot \nabla v \ d\boldsymbol{x}}_{a_0(u,v)} \ + \ \underbrace{\mu_0}_{\Theta^{a}_1(\boldsymbol{\mu})}\underbrace{\int_{\Omega_1}\nabla u \cdot \nabla v \ d\boldsymbol{x}}_{a_1(u,v)}  ,$$
$$f(v; \boldsymbol{\mu}) = \underbrace{\mu_1}_{\Theta^{f}_0(\boldsymbol{\mu})} \underbrace{\int_{\Gamma_{base}}v \ ds}_{f_0(v)}.$$


Let's set up the problem with affine dependency!

In [ ]:
# Define the domain as a rectangle, with a circle at its interior
domain = Rectangle(Point(0, 0), Point(1, 1))

center = Circle(Point(0.5, 0.5), 0.25)

domain.set_subdomain(1, center)

mesh = generate_mesh(domain, 60)
plot(mesh)
plt.show()

# Define a measure (the object that we use to integrate) that is aware of the subdomains
markers = MeshFunction('size_t', mesh, 2, mesh.domains())
dx = Measure('dx', domain=mesh, subdomain_data=markers)
ppp=plot(markers)
plt.colorbar(ppp)
plt.show()

# Reference finite element
V_element = FiniteElement('Lagrange', triangle, 1)

# Finite element space on the mesh
V = FunctionSpace(mesh, V_element)

# Trial, Test and solution functions
u = TrialFunction(V)
u_sol = Function(V)
v = TestFunction(V)


# Top Dirichlet homogeneous boundary conditions (for non-homogeneous better using a lift function)
top_BC = DirichletBC(V, 0.,
                      "on_boundary && \
                      (x[1]>1.0-DOLFIN_EPS)")


# Setting the markers for the boundaries I'm interested in for neumann BCs
boundary_markers = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)

# I set all markers to 0 in all boundaries
boundary_markers.set_all(0)

# Define the bottom boundary
bottom_boundary = AutoSubDomain(lambda x : near(x[1], 0.0, DOLFIN_EPS))

# Set its marker to 1
bottom_boundary.mark(boundary_markers, 1)

# Now I can redefine the border measure using these markers
ds = Measure("ds")(subdomain_data=boundary_markers)

# Then I can integrate only over the bottom boundary the functions that I want and assemble the RHS
g=Constant(1.)
rhs =  g*v*ds(1)
RHS=[assemble(rhs)]


# Assemble the different LHS
lhs=[]
LHS=[]

for i in range(2):
  lhs.append(inner(grad(u),grad(v))*dx(i) )
  LHS.append(assemble(lhs[i]))



Solve for 1 parameter $\mu_0=0.1$ and $\mu_1=1.$

In [ ]:
import numpy as np
import time

# Applying the Dirichlet BC (i.e. putting to diagonal 1 the rows that are related to those dofs and 0 the RHS entry related to those dofs)
top_BC.apply(LHS[0])
top_BC.apply(RHS[0])

mu = np.array([0.1,1.])

# Solve the problem for a given parameter
tic = time.time()
solve(LHS[0]+mu[0]*LHS[1], u_sol.vector(), mu[1]*RHS[0])
toc = time.time()-tic

print("Computed ROM for mu = (%1.3f,%1.3f) in %1.3e sec"%(mu[0],mu[1],toc))

# Plot the solution
plt.figure()
pp=plot(u_sol)
plt.colorbar(pp)



## 4. Generate a reduced space

We use the proper orthogonal decomposition (POD) to extract a low dimensional linear subspace of $V_h$ that we call $V_{RB}$.

The POD needs some *snapshots* that are solutions for different parameters $\boldsymbol{\mu}_i \in \mathcal{P}$ for $i=1,\dots,N_{train}$ that we denote by $u(\boldsymbol{\mu}_i)$, we store them in a matrix $X_{train} \in \mathbb R^{N_{train}\times N_h}$ and we apply the singular value decomposition (SVD) of this matrix (the economic version, `full_matrix=False`).

This gives us that
$$
L S R = X_{train} ,\qquad \text{ with }L,S\in \mathbb{R}^{N_{train}\times N_{train}}, R\in \mathbb{R}^{N_{train}\times N_{h}}
$$
where $S$ is a diagonal matrix that contains the singular values of $X_{train}$ ordered from the largest to the smallest, while $L$ is an orthonormal matrix and the columns of $R$ are orthonormal vectors.

Then, we can approximate $X_{train}$ retaining only the $N_{RB}$ most important modes, keeping only the first $N_{RB}$ singular values and related modes, i.e.,
$$
\begin{align*}
&\tilde{L}:=L[:,:N_{RB}] \in \mathbb{R}^{N_{train}\times N_{RB}},\qquad
\tilde{S}:=S[:N_{RB},:N_{RB}] \in \mathbb{R}^{N_{RB}\times N_{RB}},\qquad
\tilde{R}:=R[:N_{RB},:] \in \mathbb{R}^{N_{RB}\times N_h},\\
&X_{train} \approx \tilde{L}\tilde{S}\tilde{V}.
\end{align*}
$$
If the decay of the singular values is fast (exponential) and $N_{RB}$ is sufficiently large, we are not making a large error!

So, we can write for every $u(\boldsymbol{\mu}) \approx \sum_{i=1}^{N_{RB}} \hat u_i(\boldsymbol{\mu}) \psi_i$ where $\psi_i$ are the column of $\tilde{R}$.




In [ ]:
import numpy as np
import time

top_BC.apply(LHS[0])
top_BC.apply(RHS[0])

mu_min = np.array([0.1,-1.])
mu_max = np.array([10.,1.])

N_train = 100
training_set = np.random.rand(N_train,2)*(mu_max-mu_min)+mu_min
U = np.zeros((N_train, V.dim()))
times = np.zeros(N_train)

for i, mu in enumerate(training_set):
  tic = time.time()
  solve(LHS[0]+mu[0]*LHS[1], u_sol.vector(), mu[1]*RHS[0])
  times[i] = time.time()-tic

  print("Computed FOM for mu = (%1.3f,%1.3f) in %1.3e sec"%(mu[0],mu[1],times[i]))
  # plt.figure()
  # pp=plot(u_sol)
  # plt.colorbar(pp)

  U[i,:] = u_sol.vector()[:]

L_svd,S_svd,R_svd=np.linalg.svd(U, full_matrices=False)

In [ ]:
print("shape of L_svd", np.shape(L_svd))
print("shape of S_svd", np.shape(S_svd))
print("shape of R_svd", np.shape(R_svd))

# Is V orthonormal?
print("Difference between V V^T -I")
print(np.linalg.norm(R_svd@R_svd.T-np.eye(N_train)))

# Is L orthonormal
print("Difference between L L^T -I")
print(np.linalg.norm(L_svd@L_svd.T-np.eye(N_train)))

plt.semilogy(S_svd,'o')

In [ ]:
n_RB = 2
VRB = R_svd[:n_RB,:]

## Plotting some basis functions

In [ ]:
uaux = Function(V)
for i in range(np.minimum(5,n_RB)):
  uaux.vector()[:]=VRB[i][:]
  plt.figure()
  ppp= plot(uaux)
  plt.colorbar(ppp)
  plt.title("Basis #%d"%i)

## 5. Assemble the reduced system
Now that we have our reduced space given by the span of the reduced basis $\psi_i$ and we store these vectors in the matrix $V_{RB} \in \mathbb{R}^{N_{RB}\times N_h}$.

Now, we can do a Galerkin projection of the full problem of dimension $N_h$ on the reduced space $V_{RB}$, i.e., we can use $u_{RB} := \sum_{i=1}^{N_{RB}} \hat{u}_i \psi_{i}$ and test with the RB functions, i.e., we want to solve the linear system
$$
a(\psi_j,\psi_i)\hat{u}_i = F(\psi_i),\qquad \forall i=1,\dots,N_{RB},
$$
that, for the affine dependency of the parameters becomes
$$
\sum_{k=1}^{K_a} \Theta^a_{k}(\boldsymbol{\mu})a_k(\psi_j,\psi_i)\hat{u}_i(\boldsymbol{\mu}) = \sum_{k=1}^{K_F} \Theta^F_k (\boldsymbol{\mu}) F_k(\psi_i),\qquad \forall i=1,\dots,N_{RB},
$$
and in matrix form, it becomes
$$
\left( \sum_{k=1}^{K_a} \Theta^a_{k}(\boldsymbol{\mu})\hat{A}_k \right) \hat{u}(\boldsymbol{\mu}) = \sum_{k=1}^{K_F} \Theta^F_k (\boldsymbol{\mu}) \hat{F}_k,\qquad \text{with } \hat{A}_k = V_{RB} A_k V_{RB}^T \in \mathbb R^{N_{RB}\times N_{RB}} \text { and } \hat{F}_k  = V_{RB}F_k\in \mathbb R ^{N_{RB}}.
$$


In [ ]:
# Computing the reduced matrices and vectors

LHS_RB = []
RHS_RB = []

# I need two auxiliary functions to store basis functions and the results of Matrix vector multiplication
# With the dolfin matrix type I can only do matrix vector multiplication, so I cannot do matrix x matrix
uaux = Function(V)
u_mult = Function(V)


# Loop over the affine terms of the LHS
for n in range(len(LHS)):
  # Initialize the reduced matrix (full but small)
  LHS_RB.append(np.zeros((n_RB,n_RB)))

  # Loop over the trial functions (reduced basis functions)
  for j in range(n_RB):
    uaux.vector()[:] = VRB[j,:]

    # Computing U_aux =A_n * psi_j
    LHS[n].mult(uaux.vector(),u_mult.vector())

    # Computing \hat{A}_{ij} = (psi_i, U_aux)
    for i in range(n_RB):
      LHS_RB[n][i,j] = np.dot(VRB[i,:],u_mult.vector())





In [ ]:
# Computing the reduced RHS
for n in range(len(RHS)):
  RHS_RB.append(np.zeros(n_RB))
  for j in range(n_RB):
    RHS_RB[n][j] = np.dot(VRB[j,:],RHS[n])


## 6. Solve for a new parameter
Given a new test parameter I want to solve the reduced problem.

In [ ]:
# Create a class to solve Full order model (FOM) and reduced order model (ROM)
class Poisson:
  def __init__(self,V,VRB,LHS,RHS, LHS_RB, RHS_RB):
    self.V = V
    self.VRB = VRB
    self.LHS = LHS
    self.RHS = RHS
    self.LHS_RB = LHS_RB
    self.RHS_RB = RHS_RB
    self.u_sol = Function(self.V)
    self.u_RB = Function(self.V)
    self.u_RB_coeff = np.zeros(self.VRB.shape[0])
  def set_mu(self,mu):
      self.mu = mu
  def solve_FOM(self):
    # Solve the problem for a given parameter
    tic = time.time()
    solve(self.LHS[0]+self.mu[0]*self.LHS[1], self.u_sol.vector(), self.mu[1]*self.RHS[0])
    toc = time.time()-tic
    self.computational_time_FOM = toc
    return u_sol
  def solve_ROM(self):
    # Solve the problem for a given parameter
    tic = time.time()
    self.u_RB_coeff = np.linalg.solve(LHS_RB[0]+self.mu[0]*LHS_RB[1],self.mu[1]*RHS_RB[0])
    toc = time.time()-tic
    self.computational_time_ROM = toc
    return self.u_RB_coeff
  def reconstruct_ROM(self):
    self.u_RB.vector()[:] = self.u_RB_coeff@self.VRB
    return self.u_RB
  def compute_error(self):
    return assemble(inner(self.u_RB-self.u_sol,self.u_RB-self.u_sol)*dx)/assemble(inner(self.u_sol,self.u_sol)*dx)


# Test the class
poisson = Poisson(V,VRB,LHS,RHS,LHS_RB,RHS_RB)

mu = np.array([0.121251, -0.35412])
poisson.set_mu(mu)

poisson.solve_FOM()
plt.figure()
pp=plot(poisson.u_sol)
plt.colorbar(pp)
plt.title("FOM")


poisson.solve_ROM()
poisson.reconstruct_ROM()
plt.figure()
pp=plot(poisson.u_RB)
plt.colorbar(pp)
plt.title("ROM")

plt.figure()
pp=plot(poisson.u_sol-poisson.u_RB)
plt.colorbar(pp)
plt.title("Error")

print("Comp time FOM", poisson.computational_time_FOM)
print("Comp time ROM", poisson.computational_time_ROM)



## 7. Test on a large test set

In [ ]:
N_test = 500
test_set = np.random.rand(N_test,2)*(mu_max-mu_min)+mu_min

errors = np.zeros(N_test)
speed_up = np.zeros(N_test)

for i, mu in enumerate(test_set):
  poisson.set_mu(mu)
  poisson.solve_FOM()
  poisson.solve_ROM()
  poisson.reconstruct_ROM()
  poisson.compute_error()
  errors[i] = poisson.compute_error()
  speed_up[i] = poisson.computational_time_FOM/poisson.computational_time_ROM
  print("Solved for imu = %d and mu = (%1.3f,%1.3f), err = %1.3e, speedup = %1.3f"%(i,mu[0],mu[1],errors[i], speed_up[i]))


print("Mean error ", np.mean(errors))
print("Mean speed up ", np.mean(speed_up))

## Exercises
1. Plot the average error as a function of the number of basis functions in the reduced space ($N_{RB}$). Here, the FEM expensive solutions can be computed only once, while the reduced solutions will be computed multiple times for different number of reduced basis functions for all parameters in the test set. Change the Poisson class so that it can set a different reduced basis passing only the number of basis functions.
2. Add more parameters to the problem (more subdomains, more boundary conditions)
3. Try with nonhomogeneous Dirichlet BC (we need a lifting function)
4. Try more complex geometries